Ok on va commencer par load un seul fichier JSON et les GTFS aussi

In [160]:
#Fichier GTFS

import gtfs_functions as gtfs
import matplotlib
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs("gtfs3Sept.zip",busiest_date=False)
stop_freq = gtfs.stops_freq(stop_times, stops)
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes)

In [ ]:
)
stop_freq = gtfs.stops_freq(stop_times, stops)
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes)

In [2]:
#Fichier JSON 

import json
import pandas as pd
import numpy as np

with open('vehiclePosition10.json') as f:
    d=json.load(f)

data = pd.json_normalize(d['data'])

In [3]:
#We start cleaning 

# Here I clean every ApiCall (i.e. for each time) that contains nothing or is assigned to nothing (None)

data_cleaned = [ApiCall for ApiCall in d['data'] 
if ((ApiCall is not None) & (ApiCall['Responses'] is not None ) &(ApiCall['Responses'] is not [] )) 
]

# Then we go to clean the 'Responses' to remove the None value

for i,ApiCall in enumerate(data_cleaned):
    data_cleaned[i]['Responses'] = [response for response in ApiCall['Responses'] if (response is not None) ]

# Once it's done, we go to the 'lines' key and we again remove the None value

for i,ApiCall in enumerate(data_cleaned):
    data_cleaned[i]['Responses'] = [response for response in ApiCall['Responses'] 
    if (response['lines'] is not None) 
    ]

vehiclePosition10 = pd.json_normalize(data=data_cleaned,record_path=['Responses','lines','vehiclePositions'],
meta=['time',['Responses','lines','lineId']],errors='ignore')

vehiclePosition10.columns = ['directionId', 'distanceFromPoint','pointId','time','lineId']

#Change the name of the column so that it's easy

In [258]:
#Important functions

import pandas as pd
import datetime

calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')

def CheckRange(Start,End,date):
    return int(date) in range(int(Start),int(End))

def FromDayToService(date):
    #date est un tuple ('20210907','tuesday')
    ListServiceId=[]
    for i in range(len(calendardf)):

        startdate,endate=calendardf.iloc[i,[8,9]]
        
        if CheckRange(startdate,endate+1,date[0]):
            if calendardf.loc[i,date[1]]==1:
                
                ListServiceId.append(str(calendardf.iloc[i,0]))

    ModifServiceId= calendarDatesdf[calendarDatesdf['date']==int(date[0])][['service_id','exception_type']]
    print('Number of service_id involved: '+ str(len(ListServiceId)))
    for modif in ModifServiceId.iterrows():
        service = str(modif[1]['service_id'])
        exception = modif[1]['exception_type']
        if exception == 1:
            ListServiceId.append(service)
            print('ServiceId: '+str(service)+' correclty added' )
        elif (exception == 2) & (service in ListServiceId):
            ListServiceId.remove(service)
            print('ServiceId: '+str(service)+' correclty removed' )
        elif (exception == 2) & (service not in ListServiceId):
            print('Flag : exception 2 but no service_id = ' +service+ ' find in calendar')
    return (ListServiceId)

def FromLineIdtoRouteId(LineId):
    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def SelectInGTFS(day,lineId,stopid):
    ListOfService = FromDayToService(day)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopid)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    Comparaisondf=Comparaisondf[['trip_id','arrival_time','stop_id','route_id','service_id']]
    ArrivalTime = Comparaisondf['arrival_time'].values
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Comparaisondf['Interval']= IntervalTime
    Comparaisondf['Ponctuality']=np.array(IntervalTime)>720
    return Comparaisondf

def ScheduledTime(date,lineId,stopId):
    ListOfService = pd.Series(FromDayToService(date))
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Comparaisondf['arrival_time'].values
    print('Number of service_id involved: '+ str(len(ListOfService)))
    Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    Schedule.insert(0,ArrivalTime[0])
    Punctuality = np.array(Schedule)>720 
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            Schedule[i]=ArrivalTime[i]
    Schedule = list(zip(Schedule,Punctuality))

    return Schedule

def actual_timePierre(lineId, stopId, date):
    year = int(date[0][:4])
    month = int(date[0][4:6])
    day = int(date[0][6:8]) 
    ListOfService = FromDayToService(date)
    RouteId = FromLineIdtoRouteId(lineId)
    date_init = datetime.datetime(year, month, day, 4)
    epoch_date_init = float(datetime.datetime.timestamp(date_init))
    print(epoch_date_init)
    date_end = datetime.datetime(year, month, day+1, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
    epoch_date_end = float(datetime.datetime.timestamp(date_end))
    print(epoch_date_end)
    listvehiculearrivals = []
    df = vehiclePosition10[(vehiclePosition10['time'].between(epoch_date_init,epoch_date_end))
                            &(vehiclePosition10['lineId']==lineId)
                            &(vehiclePosition10['pointId']==stopId)
                            &(vehiclePosition10['distanceFromPoint'])]
    
    return df

In [269]:
stops[stops['stop_name']=='PETILLON']
#routes[routes['route_short_name']=='7']

,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry
1064,2698,NaN,PETILLON,NaN,NaN,NaN,0,NaN,POINT (4.40333 50.82973)
1959,5312,NaN,PETILLON,NaN,NaN,NaN,0,NaN,POINT (4.40232 50.82930)
1964,5352,NaN,PETILLON,NaN,NaN,NaN,0,NaN,POINT (4.40280 50.82955)
2462,7968B,NaN,PETILLON,NaN,NaN,NaN,0,NaN,POINT (4.40296 50.83002)
2497,8211,NaN,PETILLON,NaN,NaN,NaN,0,23,POINT (4.40386 50.82793)
2498,8212,NaN,PETILLON,NaN,NaN,NaN,0,23,POINT (4.40399 50.82746)


In [276]:
stop_times[(stop_times['stop_id']=='5312')&(stop_times['direction_id']==0)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,route_id,service_id,direction_id,shape_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry


In [272]:
Date=('20210910','friday')
lineid = '7'
stopid = '5311'


len(ScheduledTime(Date,lineid,stopid))


Number of service_id involved: 97
ServiceId: 237231071 correclty removed
ServiceId: 237381031 correclty removed
ServiceId: 238264040 correclty removed
ServiceId: 237429031 correclty removed
ServiceId: 237137031 correclty removed
ServiceId: 237117001 correclty removed
ServiceId: 236786030 correclty removed
ServiceId: 237167030 correclty removed
ServiceId: 237113030 correclty removed
ServiceId: 237254070 correclty removed
ServiceId: 237091031 correclty removed
ServiceId: 237076031 correclty removed
ServiceId: 237587001 correclty removed
ServiceId: 237578031 correclty removed
ServiceId: 237505031 correclty removed
ServiceId: 237502031 correclty removed
ServiceId: 236961060 correclty removed
ServiceId: 237580031 correclty removed
ServiceId: 238178032 correclty removed
Number of service_id involved: 78


158

In [196]:
calendarDatesdf[(calendarDatesdf['exception_type']==2)&(calendarDatesdf['date']==20210912)]
calendarDatesdf[(calendarDatesdf['exception_type']==2)].head(30)

,service_id,date,exception_type
6,233898050,20210830,2
12,235811052,20210830,2
13,237231071,20210902,2
14,237231071,20210903,2
15,237231071,20210906,2
16,237231071,20210907,2
17,237231071,20210909,2
18,237231071,20210910,2
19,237231071,20210913,2
20,237231071,20210914,2


In [168]:
ServiceWkd = calendardf[(calendardf['saturday']==1) | (calendardf['sunday']==1)]['service_id']
ServiceWeek = calendardf[(calendardf['monday']==1) 
                        | (calendardf['tuesday']==1)
                        | (calendardf['wednesday']==1)
                        | (calendardf['thursday']==1)
                        | (calendardf['friday']==1)]['service_id']
                        


stop_times[stop_times['service_id'].astype('int64').isin(ServiceWkd)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,route_id,service_id,direction_id,shape_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry
99915,112401816238195506,27960.0,27960.0,1732,1,0,1,20,238195506,1,013b0067,NaN,UZ-VUB,NaN,NaN,NaN,0,NaN,POINT (4.30895 50.88571)
99916,112401816238195506,28006.0,28006.0,1743,2,0,0,20,238195506,1,013b0067,NaN,UZ BRUSSEL,NaN,NaN,NaN,0,NaN,POINT (4.31145 50.88673)
99917,112401816238195506,28058.0,28058.0,1744,3,0,0,20,238195506,1,013b0067,NaN,UZ-PEDIATRIE,NaN,NaN,NaN,0,NaN,POINT (4.30940 50.88970)
99918,112401816238195506,28145.0,28145.0,1334,4,0,0,20,238195506,1,013b0067,NaN,ARBRE BALLON,NaN,NaN,NaN,0,NaN,POINT (4.31401 50.89262)
99919,112401816238195506,28208.0,28208.0,1745,5,0,0,20,238195506,1,013b0067,NaN,DEMUNTER,NaN,NaN,NaN,0,NaN,POINT (4.31677 50.88968)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820504,113028649236519600,26940.0,26940.0,6427F,17,0,0,8,236519600,1,097t0371,NaN,JANSON,NaN,NaN,NaN,0,NaN,POINT (4.35435 50.82545)
2820505,113028649236519600,27000.0,27000.0,6430F,18,0,0,8,236519600,1,097t0371,NaN,JANSON,NaN,NaN,NaN,0,NaN,POINT (4.35483 50.82614)
2820506,113028649236519600,27095.0,27095.0,5066F,19,0,0,8,236519600,1,097t0371,NaN,FAIDER,NaN,NaN,NaN,0,NaN,POINT (4.35608 50.83004)
2820507,113028649236519600,27180.0,27180.0,5068F,20,0,0,8,236519600,1,097t0371,NaN,STEPHANIE,NaN,NaN,NaN,0,NaN,POINT (4.35718 50.83356)


In [146]:
calendarDatesdf[calendarDatesdf['date']==20210912]
type(stop_times['service_id'][0])

str

In [132]:
ServicesTest= pd.Series(FromDayToService(('20210911','saturday')))
test = ServicesTest.isin(calendarDatesdf['service_id'])
test[test==True]

Series([], dtype: bool)

In [167]:
Services = pd.Series(FromDayToService(('20210905','sunday')))
stop_times[stop_times['service_id'].isin(Services)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,route_id,service_id,direction_id,shape_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry
815944,113094829236755601,88200.0,88200.0,9600B,1,0,0,18,236755601,0,012b0188,NaN,BRUSSELS AIRPORT,NaN,NaN,NaN,0,NaN,POINT (4.48176 50.89815)
815945,113094829236755601,88613.0,88613.0,3017,2,0,0,18,236755601,0,012b0188,NaN,BOURGET,NaN,NaN,NaN,0,NaN,POINT (4.43033 50.87815)
815946,113094829236755601,88740.0,88740.0,5048,3,0,0,18,236755601,0,012b0188,NaN,DA VINCI,NaN,NaN,NaN,0,NaN,POINT (4.41044 50.87615)
815947,113094829236755601,88978.0,88978.0,2695,4,0,0,18,236755601,0,012b0188,NaN,GENEVE,NaN,NaN,NaN,0,NaN,POINT (4.40324 50.86383)
815948,113094829236755601,89220.0,89220.0,2250,5,0,0,18,236755601,0,012b0188,NaN,MEISER,NaN,NaN,NaN,0,NaN,POINT (4.39779 50.85396)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801762,113248929236538600,90247.0,90247.0,6122,11,0,0,75,236538600,0,093t0578,NaN,PRINC. CLEMENTINE,NaN,NaN,NaN,0,NaN,POINT (4.35379 50.87713)
2801763,113248929236538600,90420.0,90420.0,2878F,12,0,0,75,236538600,0,093t0578,NaN,MARIE-CHRISTINE,NaN,NaN,NaN,0,NaN,POINT (4.35945 50.87373)
2801764,113248929236538600,90540.0,90540.0,2543F,13,0,0,75,236538600,0,093t0578,NaN,JULES DE TROOZ,NaN,NaN,NaN,0,NaN,POINT (4.36134 50.87076)
2801765,113248929236538600,90586.0,90586.0,6178,14,0,0,75,236538600,0,093t0578,NaN,MASUI,NaN,NaN,NaN,0,NaN,POINT (4.36252 50.86917)


In [86]:
stops[stops['stop_name']=='GRIBAUMONT']

,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry
2183,8101,NaN,GRIBAUMONT,NaN,NaN,NaN,0,35,POINT (4.41829 50.84251)
2184,8102,NaN,GRIBAUMONT,NaN,NaN,NaN,0,35,POINT (4.41897 50.84259)


In [94]:
ServicesTest= FromDayToService(('20210919','sunday'))
ServicesTest=pd.DataFrame(ServicesTest)
apd = stop_times['service_id'].isin(ServicesTest)
apd[apd==True]



20210919 sunday


Series([], Name: service_id, dtype: bool)

In [108]:
ServiceInStopTime = stop_times['service_id']
ServiceForDay = pd.DataFrame(FromDayToService(('20210912','sunday')))

ServiceForDay.isin(ServiceInStopTime )

20210912 sunday


,0
0,False
1,False
2,False
3,False
4,False
...,...
59,False
60,False
61,False
62,False


In [44]:
vehiclePosition10['time']=vehiclePosition10['time'].astype('float64')

In [51]:
actual_timePierre(lineid,stopid,Date)

1630980000.0
1631062800.0


,directionId,distanceFromPoint,pointId,time,lineId


In [6]:
copy = vehiclePosition10
copy.head()

,directionId,distanceFromPoint,pointId,time,lineId
0,8162,0,8082,1631779023468,1
1,8731,0,8162,1631779023468,1
2,8162,1,8272,1631779023468,1
3,8731,0,8121,1631779023468,1
4,8161,0,8052,1631779023468,1


In [7]:
from datetime import datetime
#df_vehiculepositions['time'].sort_values()
copy = copy.astype({'time':'int64'})
copy['time']=copy['time'].floordiv(1000)
def datefromtimestamp(timestamp):
    date = datetime.utcfromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')
    return date
time_fun = datefromtimestamp(copy['time'][0])


In [8]:
def datefromtimestamp(timestamp):
    date = datetime.utcfromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')
    return date

def FromTimeCodeToDate(JsonDataFrame):
    DataFrame = JsonDataFrame
    DataFrame = DataFrame.astype({'time':'int64'})
    DataFrame['time']=DataFrame['time'].floordiv(1000)
    print('flage')
    DataFrame['time']=[datefromtimestamp(timecode) for timecode in DataFrame['time']]

    return DataFrame
FromTimeCodeToDate(vehiclePosition10).head()

flage


,directionId,distanceFromPoint,pointId,time,lineId
0,8162,0,8082,09-16-2021 07:57:03,1
1,8731,0,8162,09-16-2021 07:57:03,1
2,8162,1,8272,09-16-2021 07:57:03,1
3,8731,0,8121,09-16-2021 07:57:03,1
4,8161,0,8052,09-16-2021 07:57:03,1


In [9]:

date = time_fun[0:10].split('-')
test=''
for item in date:
    test += item 

In [10]:
test='123456789IUAY'
test.isdecimal()

False